In [1]:
import pandas as pd
df = pd.read_csv("../data/raw/lending_club.csv", low_memory=False)
df.shape


(2260668, 145)

In [2]:
df['loan_status'].value_counts()

loan_status
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: count, dtype: int64

In [3]:
bad_status=[ "Charged Off",
    "Default",
    "Late (31-120 days)",
    "Late (16-30 days)",
    "Does not meet the credit policy. Status:Charged Off"]
df["default_label"]=df["loan_status"].apply(lambda x:1 if x in bad_status else 0)
df=df.copy()

C:\Users\HP\AppData\Local\Temp\ipykernel_6792\488268381.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["default_label"]=df["loan_status"].apply(lambda x:1 if x in bad_status else 0)


In [4]:
df['default_label'].value_counts()

default_label
0    1972587
1     288081
Name: count, dtype: int64

In [5]:
df['default_label'].value_counts(normalize=True)*100

default_label
0    87.25682
1    12.74318
Name: proportion, dtype: float64

In [6]:
missing_percent = df.isnull().mean().sort_values(ascending=False) * 100
missing_percent.head(15)


id                                            100.000000
member_id                                     100.000000
url                                           100.000000
orig_projected_additional_accrued_interest     99.627278
deferral_term                                  99.530537
hardship_start_date                            99.530537
hardship_loan_status                           99.530537
hardship_payoff_balance_amount                 99.530537
hardship_end_date                              99.530537
hardship_type                                  99.530537
hardship_last_payment_amount                   99.530537
hardship_amount                                99.530537
hardship_status                                99.530537
hardship_reason                                99.530537
hardship_dpd                                   99.530537
dtype: float64

In [7]:
df.columns


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_last_payment_amount', 'disbursement_method',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term', 'default_label'],
      dtype='str', length=146)

In [8]:
for col in df.columns:
    print(col)
    

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
b

In [9]:
# Columns to drop (leakage, IDs, admin, sparse, post-loan)
drop_cols = [
    # Identifiers & text noise
    "id", "member_id", "url", "zip_code", "title", "desc",

    # Post-loan payment & outcome (LEAKAGE)
    "funded_amnt", "funded_amnt_inv",
    "out_prncp", "out_prncp_inv",
    "total_pymnt", "total_pymnt_inv",
    "total_rec_prncp", "total_rec_int",
    "total_rec_late_fee",
    "recoveries", "collection_recovery_fee",
    "last_pymnt_d", "last_pymnt_amnt",
    "next_pymnt_d", "last_credit_pull_d",

    # Policy / admin / meta
    "policy_code", "initial_list_status",
    "pymnt_plan", "application_type",
    "disbursement_method",

    # Joint applicant features
    "annual_inc_joint", "dti_joint",
    "verification_status_joint", "revol_bal_joint",
    "sec_app_earliest_cr_line", "sec_app_inq_last_6mths",
    "sec_app_mort_acc", "sec_app_open_acc",
    "sec_app_revol_util", "sec_app_open_act_il",
    "sec_app_num_rev_accts",
    "sec_app_chargeoff_within_12_mths",
    "sec_app_collections_12_mths_ex_med",
    "sec_app_mths_since_last_major_derog",

    # Hardship & settlement (post-distress)
    "hardship_flag", "hardship_type", "hardship_reason",
    "hardship_status", "deferral_term", "hardship_amount",
    "hardship_start_date", "hardship_end_date",
    "payment_plan_start_date", "hardship_length",
    "hardship_dpd", "hardship_loan_status",
    "orig_projected_additional_accrued_interest",
    "hardship_payoff_balance_amount",
    "hardship_last_payment_amount",
    "debt_settlement_flag", "debt_settlement_flag_date",
    "settlement_status", "settlement_date",
    "settlement_amount", "settlement_percentage",
    "settlement_term",

    # Non-feature columns
    "loan_status"   # original text label (we use default_label instead)
]


In [10]:
df_model=df.drop(columns=drop_cols,errors="ignore")

In [11]:
print("Original Columns:",df.shape[1])
print("Model columns:", df_model.shape[1])

Original Columns: 146
Model columns: 83


In [12]:
for col in df_model.columns:
    print(col)

loan_amnt
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
purpose
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
collections_12_mths_ex_med
mths_since_last_major_derog
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_bc_dlq
mths_since_recent_inq
mths_since_recent_revol_delinq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m

In [13]:
"default_label" in df_model.columns

True

In [14]:
df_model["default_label"].value_counts()


default_label
0    1972587
1     288081
Name: count, dtype: int64

In [15]:
print("Rows:", df_model.shape[0])
print("Columns:", df_model.shape[1])


Rows: 2260668
Columns: 83


In [16]:
leakage_check = [
    col for col in df_model.columns 
    if "pymnt" in col or "recover" in col or "hardship" in col
]

leakage_check


[]

In [17]:
df_model.dtypes.value_counts()


float64    69
str        11
int64       3
Name: count, dtype: int64

In [18]:
df_model.describe(include="all").T.head(10)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
loan_amnt,2260668.0,NaN,NaN,NaN,15046.931228,9190.245488,500.0,8000.0,12900.0,20000.0,40000.0
term,2260668,2,36 months,1609754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_rate,2260668.0,NaN,NaN,NaN,13.092913,4.832114,5.31,9.49,12.62,15.99,30.99
installment,2260668.0,NaN,NaN,NaN,445.807646,267.173725,4.93,251.65,377.99,593.32,1719.83
grade,2260668,7,B,663557,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_grade,2260668,35,C1,145903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_title,2093699,512694,Teacher,38824,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_length,2113761,11,10+ years,748005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_ownership,2260668,6,MORTGAGE,1111450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,2260664.0,NaN,NaN,NaN,77992.428687,112696.199574,0.0,46000.0,65000.0,93000.0,110000000.0


In [19]:
X = df_model.drop(columns=["default_label"])
y = df_model["default_label"]


In [20]:
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2260668, 82)
y shape: (2260668,)


In [21]:
X.dtypes

loan_amnt                       int64
term                              str
int_rate                      float64
installment                   float64
grade                             str
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 82, dtype: object

In [22]:
categorical_features = X.select_dtypes(include=["object", "string"]).columns.tolist()
numerical_features = X.select_dtypes(exclude=["object", "string"]).columns.tolist()

print("Categorical features:", categorical_features)
print("\nNumber of categorical features:", len(categorical_features))

print("\nNumerical features:", numerical_features[:10], "...")
print("Number of numerical features:", len(numerical_features))


Categorical features: ['term', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'addr_state', 'earliest_cr_line']

Number of categorical features: 11

Numerical features: ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc'] ...
Number of numerical features: 71


In [23]:
X.isnull().mean().sort_values(ascending=False).head(15) * 100


mths_since_last_record            84.112837
mths_since_recent_bc_dlq          77.011175
mths_since_last_major_derog       74.309585
mths_since_recent_revol_delinq    67.250432
mths_since_last_delinq            51.246003
il_util                           47.280273
mths_since_rcnt_il                40.250227
all_util                          38.322655
open_acc_6m                       38.313012
inq_last_12m                      38.313012
total_cu_tl                       38.313012
open_rv_12m                       38.312968
open_act_il                       38.312968
open_il_12m                       38.312968
total_bal_il                      38.312968
dtype: float64

In [24]:
from sklearn.impute import SimpleImputer

# Numerical imputer
num_imputer = SimpleImputer(strategy="median")

# Categorical imputer
cat_imputer = SimpleImputer(strategy="most_frequent")


In [25]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder(
    handle_unknown="ignore",
    min_frequency=0.01,   # rare categories grouped
    sparse_output=False
)


In [26]:
from sklearn.preprocessing import RobustScaler

num_scaler = RobustScaler()


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [28]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [29]:
numeric_pipeline = Pipeline(steps=[
    ("imputer", num_imputer),
    ("scaler", num_scaler)
])


In [30]:
categorical_pipeline = Pipeline(steps=[
    ("imputer", cat_imputer),
    ("encoder", cat_encoder)
])


In [31]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)


In [32]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


In [33]:
print("Processed train shape:", X_train_processed.shape)
print("Processed test shape:", X_test_processed.shape)


Processed train shape: (1808534, 218)
Processed test shape: (452134, 218)


In [34]:
from sklearn.linear_model import LogisticRegression


In [38]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)


In [39]:
log_reg.fit(X_train_processed, y_train)


C:\Users\HP\beneficiary-credit-scoring\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [40]:
y_test_proba = log_reg.predict_proba(X_test_processed)[:, 1]


In [41]:
y_test_pred = log_reg.predict(X_test_processed)


In [42]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC-AUC:", roc_auc)


ROC-AUC: 0.7133036824244445


In [43]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_pred)
cm


array([[253345, 141173],
       [ 19135,  38481]])

In [44]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.93      0.64      0.76    394518
           1       0.21      0.67      0.32     57616

    accuracy                           0.65    452134
   macro avg       0.57      0.66      0.54    452134
weighted avg       0.84      0.65      0.70    452134



In [45]:
import numpy as np
from sklearn.metrics import precision_score, recall_score

thresholds = np.arange(0.2, 0.61, 0.05)

print("Threshold | Recall (Default) | Precision (Default)")
print("-"*50)

for t in thresholds:
    y_pred_tuned = (y_test_proba >= t).astype(int)
    
    recall = recall_score(y_test, y_pred_tuned)
    precision = precision_score(y_test, y_pred_tuned)
    
    print(f"{t:0.2f}      | {recall:0.3f}            | {precision:0.3f}")


Threshold | Recall (Default) | Precision (Default)
--------------------------------------------------
0.20      | 0.988            | 0.136
0.25      | 0.970            | 0.144
0.30      | 0.941            | 0.154
0.35      | 0.894            | 0.166
0.40      | 0.832            | 0.180
0.45      | 0.757            | 0.196
0.50      | 0.668            | 0.214
0.55      | 0.564            | 0.235
0.60      | 0.451            | 0.259


In [36]:
final_threshold = 0.40
y_test_pred_final = (y_test_proba >= final_threshold).astype(int)


NameError: name 'y_test_proba' is not defined

In [47]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC-AUC:", roc_auc)


ROC-AUC: 0.7133036824244445


In [50]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_test_pred_final)
cm


array([[175909, 218609],
       [  9658,  47958]])

In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_test_pred_final))


              precision    recall  f1-score   support

           0       0.95      0.45      0.61    394518
           1       0.18      0.83      0.30     57616

    accuracy                           0.50    452134
   macro avg       0.56      0.64      0.45    452134
weighted avg       0.85      0.50      0.57    452134



In [ ]:
# NOTE:
# Random Forest trained on a representative subset due to computational constraints.
# Logistic Regression trained on full dataset for scalability comparison.


In [35]:
# Sample 300k rows for Random Forest training
df_rf = df_model.sample(n=300_000, random_state=42)

print("Sample shape:", df_rf.shape)



Sample shape: (300000, 83)


In [36]:
X_rf = df_rf.drop(columns=["default_label"])
y_rf = df_rf["default_label"]


In [37]:
from sklearn.model_selection import train_test_split

X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(
    X_rf,
    y_rf,
    test_size=0.2,
    stratify=y_rf,
    random_state=42
)


In [38]:
X_rf_train_processed = preprocessor.fit_transform(X_rf_train)
X_rf_test_processed = preprocessor.transform(X_rf_test)


In [41]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=14,
    min_samples_split=30,
    min_samples_leaf=10,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)


rf_model.fit(X_rf_train_processed, y_rf_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",14
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",30
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",10
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [42]:
from sklearn.metrics import roc_auc_score, classification_report

y_rf_proba = rf_model.predict_proba(X_rf_test_processed)[:, 1]
y_rf_pred = (y_rf_proba >= 0.4).astype(int)   # same threshold for fair comparison

print("RF ROC-AUC:", roc_auc_score(y_rf_test, y_rf_proba))
print(classification_report(y_rf_test, y_rf_pred))


RF ROC-AUC: 0.7273585246273028
              precision    recall  f1-score   support

           0       0.95      0.51      0.66     52365
           1       0.19      0.81      0.31      7635

    accuracy                           0.55     60000
   macro avg       0.57      0.66      0.49     60000
weighted avg       0.85      0.55      0.62     60000



In [34]:
# Sample 600k rows for Random Forest training
# checking for more improved and reliable results 
df_rf = df_model.sample(n=600_000, random_state=42)

print("Sample shape:", df_rf.shape)

X_rf = df_rf.drop(columns=["default_label"])
y_rf = df_rf["default_label"]

from sklearn.model_selection import train_test_split

X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(
    X_rf,
    y_rf,
    test_size=0.2,
    stratify=y_rf,
    random_state=42
)
X_rf_train_processed = preprocessor.fit_transform(X_rf_train)
X_rf_test_processed = preprocessor.transform(X_rf_test)

Sample shape: (600000, 83)


In [43]:


from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=14,
    min_samples_split=30,
    min_samples_leaf=10,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)


rf_model.fit(X_rf_train_processed, y_rf_train)


Sample shape: (600000, 83)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",14
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",30
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",10
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [44]:
from sklearn.metrics import roc_auc_score, classification_report

y_rf_proba = rf_model.predict_proba(X_rf_test_processed)[:, 1]
y_rf_pred = (y_rf_proba >= 0.4).astype(int)   # same threshold for fair comparison

print("RF ROC-AUC:", roc_auc_score(y_rf_test, y_rf_proba))
print(classification_report(y_rf_test, y_rf_pred))

RF ROC-AUC: 0.7380944482627563
              precision    recall  f1-score   support

           0       0.95      0.48      0.64    104642
           1       0.19      0.84      0.31     15358

    accuracy                           0.53    120000
   macro avg       0.57      0.66      0.48    120000
weighted avg       0.86      0.53      0.60    120000



In [45]:
from sklearn.ensemble import HistGradientBoostingClassifier


In [46]:
hgb_model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=6,
    max_iter=300,
    min_samples_leaf=20,
    l2_regularization=1.0,
    early_stopping=True,
    random_state=42
)


In [47]:
hgb_model.fit(X_rf_train_processed, y_rf_train)


,"loss loss: {'log_loss'}, default='log_loss'The loss function to use in the boosting process.For binary classification problems, 'log_loss' is also known as logistic loss,binomial deviance or binary crossentropy. Internally, the model fits one treeper boosting iteration and uses the logistic sigmoid function (expit) asinverse link function to compute the predicted positive class probability.For multiclass classification problems, 'log_loss' is also known as multinomialdeviance or categorical crossentropy. Internally, the model fits one tree perboosting iteration and per class and uses the softmax function as inverse linkfunction to compute the predicted probabilities of the classes.",'log_loss'
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.05
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees for binary classification. For multiclassclassification, `n_classes` trees per iteration are built.",300
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",6
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",20
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",1.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide `... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 1.4 Added `""from_dtype""` option... versionchanged:: 1.6 The default value changed from `None` to `""from_dtype""`.",'from_dtyp

In [50]:
y_hgb_proba = hgb_model.predict_proba(X_rf_test_processed)[:, 1]
y_hgb_pred = (y_hgb_proba >= 0.4).astype(int)

from sklearn.metrics import roc_auc_score

roc_auc_hgb = roc_auc_score(y_rf_test, y_hgb_proba)
print("HistGradientBoosting ROC-AUC:", roc_auc_hgb)


HistGradientBoosting ROC-AUC: 0.7630643529695585


In [51]:
from sklearn.metrics import confusion_matrix

cm_hgb = confusion_matrix(y_rf_test, y_hgb_pred)
cm_hgb


array([[103377,   1265],
       [ 14186,   1172]])

In [52]:
from sklearn.metrics import classification_report

print(classification_report(y_rf_test, y_hgb_pred))


              precision    recall  f1-score   support

           0       0.88      0.99      0.93    104642
           1       0.48      0.08      0.13     15358

    accuracy                           0.87    120000
   macro avg       0.68      0.53      0.53    120000
weighted avg       0.83      0.87      0.83    120000



In [53]:
import numpy as np
from sklearn.metrics import precision_score, recall_score

thresholds = np.arange(0.05, 0.41, 0.05)

print("Threshold | Recall (Default) | Precision (Default)")
print("-"*50)

for t in thresholds:
    y_pred_tuned = (y_hgb_proba >= t).astype(int)
    recall = recall_score(y_rf_test, y_pred_tuned)
    precision = precision_score(y_rf_test, y_pred_tuned)
    print(f"{t:0.2f}      | {recall:0.3f}            | {precision:0.3f}")


Threshold | Recall (Default) | Precision (Default)
--------------------------------------------------
0.05      | 0.960            | 0.167
0.10      | 0.837            | 0.206
0.15      | 0.676            | 0.252
0.20      | 0.503            | 0.300
0.25      | 0.345            | 0.344
0.30      | 0.224            | 0.385
0.35      | 0.138            | 0.429
0.40      | 0.076            | 0.481


In [35]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb_model = HistGradientBoostingClassifier(
    max_depth=8,
    learning_rate=0.05,
    max_iter=300,
    min_samples_leaf=30,        # better than 50 for recall
    l2_regularization=1.0,
    early_stopping=True,
    random_state=42
)



In [36]:
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(
    class_weight="balanced",
    y=y_rf_train
)


In [37]:
hgb_model.fit(
    X_rf_train_processed,
    y_rf_train,
    sample_weight=sample_weights
)


,"loss loss: {'log_loss'}, default='log_loss'The loss function to use in the boosting process.For binary classification problems, 'log_loss' is also known as logistic loss,binomial deviance or binary crossentropy. Internally, the model fits one treeper boosting iteration and uses the logistic sigmoid function (expit) asinverse link function to compute the predicted positive class probability.For multiclass classification problems, 'log_loss' is also known as multinomialdeviance or categorical crossentropy. Internally, the model fits one tree perboosting iteration and per class and uses the softmax function as inverse linkfunction to compute the predicted probabilities of the classes.",'log_loss'
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.05
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees for binary classification. For multiclassclassification, `n_classes` trees per iteration are built.",300
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",8
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",30
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",1.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide `... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 1.4 Added `""from_dtype""` option... versionchanged:: 1.6 The default value changed from `None` to `""from_dtype""`.",'from_dtyp

In [38]:
y_hgb_proba = hgb_model.predict_proba(X_rf_test_processed)[:, 1]


In [39]:
y_hgb_pred = (y_hgb_proba >= 0.4).astype(int)


In [40]:
from sklearn.metrics import roc_auc_score, classification_report

print("ROC-AUC:", roc_auc_score(y_rf_test, y_hgb_proba))
print(classification_report(y_rf_test, y_hgb_pred))


ROC-AUC: 0.7644980507510959
              precision    recall  f1-score   support

           0       0.96      0.50      0.66    104642
           1       0.20      0.86      0.33     15358

    accuracy                           0.54    120000
   macro avg       0.58      0.68      0.49    120000
weighted avg       0.86      0.54      0.61    120000



In [41]:
import numpy as np
from sklearn.metrics import precision_score, recall_score

thresholds = np.arange(0.3, 0.71, 0.05)

print("Threshold | Recall | Precision")
print("-" * 40)

for t in thresholds:
    y_pred_t = (y_hgb_proba >= t).astype(int)
    recall = recall_score(y_rf_test, y_pred_t)
    precision = precision_score(y_rf_test, y_pred_t)
    print(f"{t:0.2f}      | {recall:.3f} | {precision:.3f}")


Threshold | Recall | Precision
----------------------------------------
0.30      | 0.936 | 0.178
0.35      | 0.904 | 0.188
0.40      | 0.862 | 0.201
0.45      | 0.807 | 0.217
0.50      | 0.738 | 0.235
0.55      | 0.657 | 0.257
0.60      | 0.560 | 0.284
0.65      | 0.448 | 0.315
0.70      | 0.330 | 0.353


In [42]:
FINAL_THRESHOLD = 0.45
y_final_pred = (y_hgb_proba >= FINAL_THRESHOLD).astype(int)


In [ ]:
#Final Evaluation Report

In [43]:
from sklearn.metrics import roc_auc_score

print("Final ROC-AUC:", roc_auc_score(y_rf_test, y_hgb_proba))

from sklearn.metrics import confusion_matrix

cm_final = confusion_matrix(y_rf_test, y_final_pred)
cm_final

from sklearn.metrics import classification_report

print(classification_report(y_rf_test, y_final_pred))



Final ROC-AUC: 0.7644980507510959
              precision    recall  f1-score   support

           0       0.95      0.57      0.71    104642
           1       0.22      0.81      0.34     15358

    accuracy                           0.60    120000
   macro avg       0.58      0.69      0.53    120000
weighted avg       0.86      0.60      0.67    120000



In [ ]:
# Final Model Comparison And Selection

In [44]:
# Logistic Regression metrics
log_reg_metrics = {
    "Model": "Logistic Regression",
    "ROC_AUC": 0.7133,
    "Recall_Default": 0.83,
    "Precision_Default": 0.18
}

# Random Forest metrics
rf_metrics = {
    "Model": "Random Forest",
    "ROC_AUC": 0.7380,
    "Recall_Default": 0.84,
    "Precision_Default": 0.19
}

# HistGradientBoosting 
hgb_metrics = {
    "Model": "HistGradientBoosting",
    "ROC_AUC": 0.7644,
    "Recall_Default": 0.81,
    "Precision_Default": 0.22
}


In [45]:
import pandas as pd

comparison_df = pd.DataFrame([
    log_reg_metrics,
    rf_metrics,
    hgb_metrics
])

comparison_df


,Model,ROC_AUC,Recall_Default,Precision_Default
0,Logistic Regression,0.7133,0.83,0.18
1,Random Forest,0.7380,0.84,0.19
2,HistGradientBoosting,0.7644,0.81,0.22


In [48]:
comparison_df.sort_values(by="ROC_AUC", ascending=False)
# As a Result, the HistGradientBoosting is selected as the final Model! 

,Model,ROC_AUC,Recall_Default,Precision_Default,Selected
2,HistGradientBoosting,0.7644,0.81,0.22,True
1,Random Forest,0.7380,0.84,0.19,False
0,Logistic Regression,0.7133,0.83,0.18,False
